Tôi sẽ sử dụng file này, sau đó import lên Kaggle và chạy để sử dụng GPU và Ram của kaggle

In [ ]:
!git clone https://github.com/VuThanhLam124/Profiling_gender_dialect.git
!apt-get install -y ffmpeg

In [ ]:
cd Profiling_gender_dialect

In [ ]:
!pip install -r requirements.txt
!pip install -q transformers==4.44.0 accelerate==0.33.0 datasets==2.21.0
!pip install -q librosa soundfile audiomentations==0.35.0 wandb safetensors

In [ ]:
# ============================================================
# FINETUNE WITH ViMD
# ============================================================
import os

ENCODER = "facebook/wav2vec2-base"
encoder_short = ENCODER.split("/")[-1]

vimd_config = f"""
model:
  name: "{ENCODER}"
  num_genders: 2
  num_dialects: 3
  dropout: 0.15
  head_hidden_dim: 256
  freeze_encoder: false

training:
  batch_size: 16
  gradient_accumulation_steps: 2
  learning_rate: 5e-5
  num_epochs: 2
  warmup_ratio: 0.1
  weight_decay: 0.01
  gradient_clip: 1.0
  lr_scheduler: "linear"
  fp16: true
  dataloader_num_workers: 2

loss:
  dialect_weight: 2.5

mlflow:
  enabled: false

data:
  source: "vimd"
  vimd_path: "/kaggle/input/vimd-dataset"

audio:
  sampling_rate: 16000
  max_duration: 5

augmentation:
  enabled: true
  prob: 0.8

output:
  dir: "/kaggle/working/output_vimd"
  save_total_limit: 1
  metric_for_best_model: "dialect_acc"

early_stopping:
  patience: 2
  threshold: 0.001

labels:
  gender:
    Male: 0
    Female: 1
    0: 0
    1: 1
  dialect:
    North: 0
    Central: 1
    South: 2

seed: 42
"""

config_path = "configs/vimd_test.yaml"
with open(config_path, "w") as f:
    f.write(vimd_config)


In [ ]:
print(f"Config: {config_path}")
print(f"Encoder: {ENCODER}")
print(f"Epochs:")
print("Starting training...")

os.system(f"python finetune.py --config {config_path}")

Eval with ViDM dataset